<a href="https://colab.research.google.com/github/Angel-ica/AIRBNB-DATASET/blob/main/Linear_Regression/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cd /content/gdrive/MyDrive/project_folder/AIRBNB-DATASET

In [ ]:
'''here, we're using GridSearchcv to tune the hyperparameters of some linear models and then return the best model and it's hyperparameters.'''
import sys
# sys.path.insert(0,"/Users/angelicaaluo/Airbnb/AIRBNB-DATASET/")
import pandas  as pd
import numpy as np
import os
import joblib
import json
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn. ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path

df= pd.read_csv('/content/gdrive/MyDrive/project_folder/AIRBNB-DATASET/airbnb-property-listings/tabular_data/clean_data.csv')
df.drop('Unnamed: 19',axis=1,inplace=True)  
# X = df.select_dtypes(include=['int','float'])
X=df[["beds","bedrooms","amenities_count","Location_rating","Cleanliness_rating","bathrooms","Value_rating"]]
X = df.select_dtypes(include=['int','float'])
y=df["Price_Night"]

print(globals().keys())
if "Beds" in globals().keys():
    del Beds

# MinMaxScaler().fit_transform(X_train)

model_hyperparam_distribution={
LinearRegression: {
    'fit_intercept': [True, False],
    # 'normalize': [True, False],
    'copy_X': [True, False],
    'n_jobs': [None, 1, 2, 3, 4],
    # 'random_state': [42, 56, 71, 93]
},

DecisionTreeRegressor:{
    'max_depth': [3,5,7,10],
    'min_samples_leaf': [10, 20, 30, 50],
    'min_samples_split': [2, 5, 10, 20],
    'max_leaf_nodes': [5, 10, 15, 20],
    'max_features': [1.0, 'sqrt', 'log2', None],
    # 'criterion': ['mse', 'mae'],
    # 'criterion' :['poisson', 'squared_error', 'absolute_error', 'friedman_mse'],
    'splitter': ['best', 'random'],
    'random_state': [42, 56, 71, 93]
},

GradientBoostingRegressor:{
    'learning_rate': [0.01, 0.1, 0.5, 1],
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [3, 5, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 20],
    #'max_leaf_nodes': [5,15, 20, 25],
    'max_features': [ 'sqrt', 'log2', None],
    #'loss': ['absolute_error', 'squared_error', 'huber', 'quantile'],
    #'random_state': [42, 56, 71, 93]
},

SGDRegressor:{
    'loss': ['squared_error', 'epsilon_insensitive', 'huber', 'squared_epsilon_insensitive'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [0.001, 0.03, 0.63, 1],
    'l1_ratio': [ 0.3, 0.7],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'max_iter': [250, 750, 500, 1000]
}

}

def tune_regression_model_hyperparameters(features, label, model_hyperparam_distribution):
    features = X
    label=y

    for md,hp in model_hyperparam_distribution.items():
        model = md()
        hyper_param=hp
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
        MinMaxScaler().fit_transform(X_train)
        grid_search =GridSearchCV(estimator=model,param_grid=hp,verbose=5,cv=6,refit=True)
        grid_search.fit(X_train,y_train) 
        tuned_model=grid_search.best_estimator_
        tuned_parameters=grid_search.best_params_
        y_pred = grid_search.predict(X_test)
        eval_metrics={
            'MAE %.2f: ':{mean_absolute_error(y_test,y_pred)},
            'MSE :': {mean_squared_error(y_test,y_pred)},
            'r2_score :' : {r2_score(y_test,y_pred)},
            'RMSE:': {mean_squared_error(y_test, y_pred)**0.5},
            'Validation RMSE:': {mean_squared_error(y_val,(grid_search.predict(X_val)))**0.5}
        }
        
        destination='/content/gdrive/MyDrive/project_folder/AIRBNB-DATASET/Linear_Regression /models/'
        if not os.path.isdir(destination):
            os.mkdir(destination)
        model_destination = f'{destination}/GridSearchcv'
        if not os.path.isdir(model_destination):
            os.mkdir(model_destination)
        model_final_path=f'{model_destination}/{model}'
        save_model(tuned_model,tuned_parameters,eval_metrics,model_final_path)
    #print(tuned_model,tuned_parameters,metrics)
    return tuned_model,tuned_parameters,eval_metrics

def save_model(model,hyper_parameters,metrics,folder):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    model_path = f"{folder}/model.joblib"
    joblib.dump(model,model_path)

    hyperparameter_path = f'{folder}/hyperparameter.json'
    with open (hyperparameter_path, 'w') as hp:
        json.dump(str(hyper_parameters),hp)

    eval_path = folder + '/metrics.json'
    with open (eval_path, 'w') as ep:
        json.dump(str(metrics),ep)

def find_best_model():
    all_metrics_files = list(Path('./models/GridSearchcv/').glob('**/*metrics.json'))
    all_metric_data=[]
    for metric_file in all_metrics_files:
        with open(metric_file,'r') as f:
            metric_data=json.load(f)
            all_metric_data.append(metric_data)
    best_score=max(all_metric_data)
    for score in all_metric_data:
        if score==best_score:
            best_tuned_model=str(metric_file).split('/')[-2]
            best_tuned_model_path= f'./models/GridSearchcv/{best_tuned_model}'
            best_model_name=joblib.load(f'{best_tuned_model_path}/model.joblib')
            with open(f'{best_tuned_model_path}/hyperparameter.json', 'r') as hp:
                best_model_hp=json.load(hp)
            with open(f'{best_tuned_model_path}/metrics.json', 'r') as score:
                best_model_score=json.load(score)
            return best_model_name,best_model_hp,best_model_score



tune_regression_model_hyperparameters(X,y,model_hyperparam_distribution)


In [3]:
'''The hyperparameters used in this cell were optimized by gridsearchcv in the above cell. 
Note that it did not return the best score but the scores from linear reg showed that the model was over fitting the data.'''

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

df= pd.read_csv('/content/gdrive/MyDrive/project_folder/AIRBNB-DATASET/airbnb-property-listings/tabular_data/clean_data.csv')
df.drop('Unnamed: 19',axis=1,inplace=True)  
X=df[["beds","bedrooms","amenities_count","Location_rating","Cleanliness_rating","bathrooms"]]
X = df.select_dtypes(include=['int','float'])
y=df["Price_Night"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.5,random_state=42)

# mx=MinMaxScaler()
# X_train=mx.fit_transform(X_train)
# X_test=mx.transform(X_test)

gd=GradientBoostingRegressor(learning_rate= 0.1, max_depth= 8, max_features= None, min_samples_leaf=10, min_samples_split= 5, n_estimators=500)
gd.fit(X_train,y_train)
y_pred=gd.predict(X_test)
eval_metrics={
            'MAE : ':{mean_absolute_error(y_test,y_pred)},
            'MSE :': {mean_squared_error(y_test,y_pred)},
            'r2_score :' : {r2_score(y_test,y_pred)},
            'RMSE:': {mean_squared_error(y_test, y_pred)**0.5},
            'Validation RMSE:': {mean_squared_error(y_val,(gd.predict(X_val)))**0.5}
        }
print(eval_metrics)


{'MAE : ': {8.748281785786913}, 'MSE :': {1402.4925315049504}, 'r2_score :': {0.9170122792731107}, 'RMSE:': {37.44986690904189}, 'Validation RMSE:': {38.3720737822272}}
